In [1]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.recommenderModel.models.RecommenderV4_1 import nonMetricBasedRecommender_v2
from src.recommenderModel.functions import backTesting,dataPreparation, gradientDescent, kFolds, modelEvaluationMethods, recommendationScore, similarityScores
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [2]:
Datax = pd.read_csv("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/Rot3 AA/Recommender phase 2/datasets/backtesting/carnpar_202404_sample")
Datac = pd.read_csv("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/Rot3 AA/Recommender phase 2/datasets/backtesting/carnpar_current_sample")


In [ ]:
Datax.columns

In [ ]:
Datax

In [3]:
nRecommendations = 6
idColumnName = 'pid'
itemColumnName = 'pol_type_desc'
k_Folds = 2
categiricalColumns = [ 'businessentityname', 'trn_grp_desc', 'prod_house_desc',
       'prod_tbl_grp_desc', 'psm_opt_desc', 
       'sky_pol_type_desc', 'product_desc', 'prod_tbl_cd', 'pa_sts_desc',
       'prem_freq_desc', 
       'pmt_method_desc',
       'age_group_bands_descriptions', 
        'employment_sector',
       'gender_description', 'marital_status', 'parent_indicator',
       'time_last_product_purchased_band', 'wealth_bonus_indicator',
       'business_only_indicator',
        'intermediated_clients',
       'death_benefit_indicator', 'district_municipality_name', 'geo_province',
       'pps_full_flag', 'total_recurring_premium_band', 
        'client_segment', 'client_total_fund_value_indicator_derivation','client_income_threshold',
       'financial_affluence_segment']
numericalColumns = [ 'dependants_over_18', 'dependants_under_18','pa_term','recur_prem', 'sngl_prem', 'age','adjusted_personal_income_amount','number_of_needs_met','credit_score']
gradientStepSize = 0.1
evaluationMethod = 'MRR4'
foldSize = 2000



# data preparation

In [4]:
itemColumnsTest,nCategoricalColumnsTest,aggTestData = dataPreparation.data_preparation(Datac.copy(),idColumnName,itemColumnName)
itemColumns,nCategoricalColumns,aggTrainingData = dataPreparation.data_preparation(Datax.copy(),idColumnName,itemColumnName)

trainingData = Datax.copy()


In [ ]:
aggTrainingData

In [5]:
len(aggTestData.columns)

747

In [6]:
len(aggTrainingData.columns)

620

In [11]:
newColumns = list(set(aggTestData.columns).difference(set(aggTrainingData.columns)))
newColumnsPrefixes = list(set([col.split('_:')[0] for col in newColumns]))

In [12]:
newColumnsPrefixes

['prod_house_desc',
 'product_desc',
 'prod_tbl_cd',
 'psm_opt_desc',
 'pol_type_desc',
 'businessentityname',
 'trn_grp_desc',
 'prem_freq_desc',
 'sky_pol_type_desc']

In [15]:
aggTrainingData.set_index(idColumnName).iloc[:,:nCategoricalColumns].columns

Index(['businessentityname_:ACHIEVEMENT AWARDS GROUP',
       'businessentityname_:IFN : SANLAM PERSONAL PORTFOLIO (SP2)',
       'businessentityname_:SANLAM PERSONAL LOANS',
       'businessentityname_:SANTAM', 'businessentityname_:SKY',
       'businessentityname_:SKY Channel',
       'businessentityname_:SPF : SANLAM CORPORATE INVESTMENTS',
       'businessentityname_:SPF : SANLAM LIFE',
       'businessentityname_:SPF : SANLAM TRUST WILLS',
       'trn_grp_desc_:Cash Withdrawal/Partial Surrender',
       ...
       'client_segment_:Mass Affluent', 'client_segment_:PPS Only',
       'client_segment_:Survivalist',
       'financial_affluence_segment_:Aspirational Achievers',
       'financial_affluence_segment_:Laboured Living',
       'financial_affluence_segment_:Luxury Living',
       'financial_affluence_segment_:Money-Conscious Majority',
       'financial_affluence_segment_:Stable Spenders',
       'financial_affluence_segment_:Unknown',
       'financial_affluence_segment_:Yea

In [38]:
jw = pd.DataFrame()

jw['categoricalColumns'] = aggTrainingData.set_index(idColumnName).iloc[:,:nCategoricalColumns].columns
jw['weigths'] = optimalJaccardWeightings

columnWeightAverage = {}
for prefix in newColumnsPrefixes:
    mask_A = jw['categoricalColumns'].str.startswith(prefix + '_:')
    columnWeightAverage[prefix] = jw[mask_A]['weigths'].mean()
jw_new = pd.DataFrame()
def ave_weight(column):
    for prefix, weigth in columnWeightAverage.items():
        if column.startswith(prefix):
            return weigth
    return 0
jw_new['categoricalColumns'] = newColumns
jw_new['weigths'] = jw_new['categoricalColumns'].apply(ave_weight)
jw_newnew = pd.concat([jw,jw_new])
jw_test = pd.DataFrame()

jw_test['categoricalColumns'] = aggTestData.set_index(idColumnName).iloc[:,:nCategoricalColumnsTest].columns
pd.merge(jw_test,jw_newnew,how="left",on='categoricalColumns')

,categoricalColumns,weigths
0,businessentityname_:ACHIEVEMENT AWARDS GROUP,1.0
1,businessentityname_:IFN : SANLAM PERSONAL PORT...,1.0
2,businessentityname_:SANLAM PERSONAL LOANS,1.0
3,businessentityname_:SANTAM,1.0
4,businessentityname_:SKY,1.0
...,...,...
726,pol_type_desc_:Term Annuities,1.0
727,pol_type_desc_:Top Cover,1.0
728,pol_type_desc_:Unspecified,1.0
729,pol_type_desc_:Whole Life,1.0


In [39]:
def aveWeightAssignment(aggTrainingData,aggTestData,idColumnName,nCategoricalColumns,optimalJaccardWeightings):
    newColumns = list(set(aggTestData.columns).difference(set(aggTrainingData.columns)))
    newColumnsPrefixes = list(set([col.split('_:')[0] for col in newColumns]))
    trainingWeights = pd.DataFrame()
    trainingWeights['categoricalColumns'] = aggTrainingData.set_index(idColumnName).iloc[:,:nCategoricalColumns].columns
    trainingWeights['weigths'] = optimalJaccardWeightings

    columnWeightAverage = {}
    for prefix in newColumnsPrefixes:
        mask_A = trainingWeights['categoricalColumns'].str.startswith(prefix + '_:')
        columnWeightAverage[prefix] = trainingWeights[mask_A]['weigths'].mean()
    aveWeightsNewColumns = pd.DataFrame()
    def ave_weight(column):
        for prefix, weigth in columnWeightAverage.items():
            if column.startswith(prefix):
                return weigth
        return 0
    aveWeightsNewColumns['categoricalColumns'] = newColumns
    aveWeightsNewColumns['weigths'] = aveWeightsNewColumns['categoricalColumns'].apply(ave_weight)
    allJaccardWeights = pd.concat([trainingWeights,aveWeightsNewColumns])

    jaccardWeightsTest = pd.DataFrame()
    jaccardWeightsTest['categoricalColumns'] = aggTestData.set_index(idColumnName).iloc[:,:nCategoricalColumnsTest].columns
    return pd.merge(jaccardWeightsTest,allJaccardWeights,how="left",on='categoricalColumns')

In [40]:
aveWeightAssignment(aggTrainingData,aggTestData,idColumnName,nCategoricalColumns,optimalJaccardWeightings)

,categoricalColumns,weigths
0,businessentityname_:ACHIEVEMENT AWARDS GROUP,1.0
1,businessentityname_:IFN : SANLAM PERSONAL PORT...,1.0
2,businessentityname_:SANLAM PERSONAL LOANS,1.0
3,businessentityname_:SANTAM,1.0
4,businessentityname_:SKY,1.0
...,...,...
726,pol_type_desc_:Term Annuities,1.0
727,pol_type_desc_:Top Cover,1.0
728,pol_type_desc_:Unspecified,1.0
729,pol_type_desc_:Whole Life,1.0


# kFolds

In [17]:
cosine_weightingsPerFold,jaccard_weightingsPerFold,optimalScoresPerFold = kFolds.k_folds_optimization( trainingData, idColumnName,itemColumnName,k_Folds, aggTrainingData, aggTestData, nRecommendations, gradientStep=gradientStepSize, foldSize=foldSize, fold_sample_size=min(1000,foldSize/2),evaluationMethod=evaluationMethod)
                   
optimalJaccardWeightings = np.mean(jaccard_weightingsPerFold,axis=0)
optimalCosineWeightings = np.mean(cosine_weightingsPerFold,axis=0)

0.0 1
E 0.0  T 1
0.0 1
E 0.0  T 1


In [19]:
optimalCosineWeightings

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [43]:
len(optimalJaccardWeightings)

604

In [44]:
def k_folds2(data,idColumnName,itemColumnName,k,backDatedData,upToDateData,nRecommendations,cosineWeights,jaccardWeights,foldSize = 1000,evaluationMethod='MRR4'):

    foldScores = []

    #training data (subset of backdated data)
    itemColumns,nCategoricalColumns, trainingData = dataPreparation.data_preparation(data.copy(),idColumnName,itemColumnName) 
    print('nCategoricalColumns',nCategoricalColumns)
    print(trainingData.shape)

    trainingData = trainingData.drop(columns = ['item_count','no. unique items','set of items']).set_index(idColumnName)
   
    for i in range(k):
        print(nCategoricalColumns,len(cosineWeights))
        
        trainedData = recommendationScore.RecommendationScores(nRecommendations, cosineWeights, jaccardWeights,itemColumns,nCategoricalColumns,trainingData.copy().sample(foldSize),itemColumnName ) #recommender function to get recommendations for the fold sample
        
        trainedDataIds = trainedData[idColumnName].drop_duplicates() #ids that have recommendations, training ids

        backtestedData,modelScoreColumn = backTesting.backtesting(backDatedData,upToDateData,trainedDataIds,idColumnName,trainedData.copy(),evaluationMethod) #backtesting on the current fold

        foldScores.append(np.mean(backtestedData[modelScoreColumn]))
        
    return foldScores

In [45]:
MRR3scoresPerFold = k_folds2( trainingData, idColumnName,itemColumnName,k_Folds, aggTrainingData, aggTestData, nRecommendations, optimalCosineWeightings, optimalJaccardWeightings, foldSize=foldSize,evaluationMethod="MRR3")

nCategoricalColumns 604
(28083, 620)
604 12
604 12


In [46]:
MRR3scoresPerFold

[0.21078719723183392, 0.18906621392190154]

In [ ]:

def k_folds_optimization(data,idColumnName,itemColumnName,k,backDatedData,upToDateData,nRecommendations,gradientStep,foldSize = 1000,fold_sample_size = 1000,evaluationMethod='MRR4'):


    foldScores = [] #list of scores for each fold
    cosine_weightings = [] #list of cosine weightings used for each fold
    jaccard_weightings = [] #list of jaccard weightings used for each fold

    #training data (subset of backdated data)
    itemColumns,nCategoricalColumns, trainingData= dataPreparation.data_preparation(data.copy(),idColumnName,itemColumnName) 

    trainingData = trainingData.drop(columns = ['item_count','no. unique items','set of items']).set_index(idColumnName)
    
    for i in range(k):
        e = 0.1 #initializing error value for gradient descent
        t = 70 #to avoid infinite loop

        trainingDataSample = trainingData.copy().sample(fold_sample_size) #sample of the fold of data used for optimization

        cosineWeights = (1)*np.array([1] *(trainingDataSample.shape[1] -nCategoricalColumns)) 
        jaccardWeights = (1)*np.array([1] *(nCategoricalColumns)) #include item info

        #--------------------optimizations of weightings------------------
        #process of finding optimal weightings on fold sample

        while e > 0.005 and t>0: #as e tend to zero the change in the "loss" value tends to zero
            cosineWeights,jaccardWeights,e = gradientDescent.gradient_descent(cosineWeights,jaccardWeights,itemColumns,nCategoricalColumns,trainingDataSample,backDatedData,upToDateData,idColumnName,gradientStep,evaluationMethod=evaluationMethod,itemColumnName=itemColumnName) 
            
            t = t-1 

        #------------------------------------------------------------------------
        #found optimal weigthings applied to the full fold for final fold scoring
        trainedData = recommendationScore.RecommendationScores(nRecommendations, cosineWeights, jaccardWeights,itemColumns,nCategoricalColumns,trainingData.copy().sample(foldSize),itemColumnName)
        trainedDataIds = trainedData[idColumnName].drop_duplicates() #trainedDataIds that have recommendations, training trainedDataIds

        backtestedData,modelScoreColumn = backTesting.backtesting(backDatedData,upToDateData,trainedDataIds,idColumnName,trainedData.copy(),evaluationMethod)

        foldScores.append(np.mean(backtestedData[modelScoreColumn])) #average model score for the fold
        cosine_weightings.append(cosineWeights)
        jaccard_weightings.append(jaccardWeights)
    return cosine_weightings,jaccard_weightings,foldScores

# recommend

In [40]:
def RecommendationScores2(nRecommendations, cosineWeights, jaccardWeights,recommendItemColumns,categoricalColumns, aggregated_df,hotOneEncodingPrefix=None):


    similarityMatrix = similarityScores.similarity_matrix(aggregated_df.copy(),'l2',categoricalColumns,cosineWeights,jaccardWeights)
    itemIndicator_df = aggregated_df[recommendItemColumns]
    itemIndicator_inverse_df = pd.DataFrame(np.logical_xor(itemIndicator_df.values,1).astype(int),columns=itemIndicator_df.columns, index=itemIndicator_df.index)
    scores = np.dot(similarityMatrix,itemIndicator_df)#recommendation score, sum of similarities for items users have (dot product)
    scaler = MinMaxScaler()
    scoresScaledAll = scaler.fit(scores)
    scoresScaledAll = scaler.transform(scores) #apply normalization (MinMax)
    scoresScaledFiltered = scoresScaledAll*itemIndicator_inverse_df #make score zero for items user already has

    scores_df = pd.DataFrame(scoresScaledFiltered,columns=itemIndicator_df.columns, index=itemIndicator_df.index)
    newColumns = {}
    del itemIndicator_df,itemIndicator_inverse_df,scores,scoresScaledFiltered,scoresScaledAll

    #adding recommendations to the aggregated dataframe
    for i in range(1, nRecommendations+1, 1):
        newColumns[f"no.{i} most recommended item"] = []
        newColumns[f"no.{i} item recommendation score"] = []
    for n in aggregated_df.index:

        sortedScores = scores_df.loc[n].sort_values()
        recommendationScores = np.array(sortedScores.values[-nRecommendations:])
        prod = np.array(sortedScores.index[-nRecommendations:])
        for i in range(nRecommendations-1,-1,-1):
            j = nRecommendations-i
            newColumns[f"no.{j} most recommended item"].append(prod[i])
            newColumns[f"no.{j} item recommendation score"].append(recommendationScores[i])

    recommendations_df = pd.DataFrame(aggregated_df.index)
    recommendations_df = recommendations_df.assign(**newColumns)

#Removing the prefix from hot-one encoding
    if hotOneEncodingPrefix is not None:
        for r in range(1,nRecommendations+1,1):
            recommendations_df[f'no.{r} most recommended item'] = recommendations_df[f'no.{r} most recommended item'].str.replace(hotOneEncodingPrefix+'_', '') #cleaning up

    return recommendations_df

In [ ]:
aggTestData.drop(columns = ['item_count','no. unique items','set of items']).set_index(idColumnName).iloc[:,nCategoricalColumns:]

,recur_prem,sngl_prem,pa_term,age,adjusted_personal_income_amount,dependants_over_18,dependants_under_18,parent_indicator,wealth_bonus_indicator,death_benefit_indicator,number_of_needs_met,credit_score
pid,,,,,,,,,,,,
3047,2406.83,0.00,1411,50.0,79696.21,1.0,0.0,1.0,1.0,1.0,3.0,709.0
22742,0.00,0.00,0,76.0,74630.28,0.0,0.0,0.0,0.0,0.0,3.0,702.0
24203,679.06,0.00,3564,37.0,88417.22,0.0,0.0,0.0,0.0,1.0,3.0,687.0
33409,3336.50,46122.03,230,46.0,36337.99,1.0,1.0,1.0,1.0,0.0,1.0,726.0
44298,2383.11,5119.64,1597,35.0,16928.78,0.0,1.0,1.0,1.0,1.0,6.0,680.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13457624,1237.72,0.00,0,30.0,14331.83,0.0,0.0,0.0,0.0,0.0,1.0,604.0
13660399,0.00,0.00,0,55.0,40380.18,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13668745,2251.05,1338319.68,0,71.0,6637.18,0.0,0.0,0.0,0.0,1.0,2.0,0.0


In [42]:
recommendationData = RecommendationScores2(nRecommendations,optimalCosineWeightings, optimalJaccardWeightings,itemColumns,nCategoricalColumns,aggTestData.drop(columns = ['item_count','no. unique items','set of items']).set_index(idColumnName),itemColumnName )

InvalidArgumentError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [604] vs. [28083,731] [Op:Mul] name: 

In [16]:
len(optimalCosineWeightings)
nCategoricalColumns

604

In [12]:
aggTestData

,pid,businessentityname_ACHIEVEMENT AWARDS GROUP,businessentityname_IFN : SANLAM PERSONAL PORTFOLIO (SP2),businessentityname_SANLAM PERSONAL LOANS,businessentityname_SANTAM,businessentityname_SKY,businessentityname_SKY Channel,businessentityname_SPF : SANLAM CORPORATE INVESTMENTS,businessentityname_SPF : SANLAM LIFE,businessentityname_SPF : SANLAM TRUST TRUSTS,...,dependants_over_18,dependants_under_18,parent_indicator,wealth_bonus_indicator,death_benefit_indicator,number_of_needs_met,credit_score,item_count,no. unique items,set of items
0,3047,0,1,0,0,0,0,0,1,0,...,1.0,0.0,1.0,1.0,1.0,3.0,709.0,3,3,"{Top Cover, IFN - Personal Portfolio, Retireme..."
1,22742,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,3.0,702.0,5,2,"{SPF - Corporate Investment, IFN - Personal Po..."
2,24203,0,1,0,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,1.0,3.0,687.0,7,3,"{Top Cover, SPF - Corporate Investment, IFN - ..."
3,33409,0,0,0,0,0,0,0,1,0,...,1.0,1.0,1.0,1.0,0.0,1.0,726.0,1,1,{Retirement Annuities :Non PPS}
4,44298,0,0,0,0,0,0,1,1,0,...,0.0,1.0,1.0,1.0,1.0,6.0,680.0,5,5,"{SPF - Corporate Investment, Endowment, SPF - ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28078,13457624,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,604.0,2,1,{SKY - Funeral Policies}
28079,13660399,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,{IFN - Personal Portfolio}
28080,13668745,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,5,3,"{Whole Life:Plan 27/Choice Term:Plan 28, IFN -..."
28081,13694274,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,2.0,623.0,3,1,{SKY - Funeral Policies}


In [13]:
aggTrainingData

,pid,businessentityname_ACHIEVEMENT AWARDS GROUP,businessentityname_IFN : SANLAM PERSONAL PORTFOLIO (SP2),businessentityname_SANLAM PERSONAL LOANS,businessentityname_SANTAM,businessentityname_SKY,businessentityname_SKY Channel,businessentityname_SPF : SANLAM CORPORATE INVESTMENTS,businessentityname_SPF : SANLAM LIFE,businessentityname_SPF : SANLAM TRUST WILLS,...,dependants_over_18,dependants_under_18,parent_indicator,wealth_bonus_indicator,death_benefit_indicator,number_of_needs_met,credit_score,item_count,no. unique items,set of items
0,3047,0,0,0,0,0,0,0,1,0,...,1.0,0.0,1.0,1.0,1.0,3.0,709.0,1,1,{Retirement Annuities :Non PPS}
1,22742,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,3.0,702.0,1,1,{IFN - Personal Portfolio}
2,24203,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,1.0,3.0,687.0,1,1,{Top Cover}
3,33409,0,0,0,0,0,0,0,1,0,...,1.0,1.0,1.0,1.0,0.0,1.0,726.0,1,1,{Retirement Annuities :Non PPS}
4,44298,0,0,0,0,0,0,0,1,0,...,0.0,1.0,1.0,1.0,1.0,6.0,680.0,1,1,{Retirement Annuities :Non PPS}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28078,13457624,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,604.0,1,1,{SKY - Funeral Policies}
28079,13660399,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1,{IFN - Personal Portfolio}
28080,13668745,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1,1,{IFN - Personal Portfolio}
28081,13694274,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,2.0,623.0,1,1,{SKY - Funeral Policies}
